<a href="https://colab.research.google.com/github/Lokhande10/Pyspark/blob/main/Pyspark_ML_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 61.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=e645ec7b201fb17237f03427cd7a0793ca2f84445da31577fccc5c432313f6d8
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [6]:
## Read The dataset
training = spark.read.csv('datasheet3.csv',header=True,inferSchema=True)

In [7]:
training.show()

+------+---+---+------+
|  Name|Age|Exp|Salary|
+------+---+---+------+
|Sachin| 25|  5| 20000|
|  Ravi| 22|  1| 50000|
| Sanju| 23|  8| 30000|
|  Viru| 27|  2| 24000|
|Saddhu| 24|  7| 25000|
|  Rutu| 25|  3| 25000|
|Nikhil| 24|  3| 29000|
|Vishal| 26|  2| 37000|
+------+---+---+------+



In [8]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Exp: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [10]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Exp"],outputCol="Independent Features")

In [11]:


output=featureassembler.transform(training)



In [12]:


output.show()



+------+---+---+------+--------------------+
|  Name|Age|Exp|Salary|Independent Features|
+------+---+---+------+--------------------+
|Sachin| 25|  5| 20000|          [25.0,5.0]|
|  Ravi| 22|  1| 50000|          [22.0,1.0]|
| Sanju| 23|  8| 30000|          [23.0,8.0]|
|  Viru| 27|  2| 24000|          [27.0,2.0]|
|Saddhu| 24|  7| 25000|          [24.0,7.0]|
|  Rutu| 25|  3| 25000|          [25.0,3.0]|
|Nikhil| 24|  3| 29000|          [24.0,3.0]|
|Vishal| 26|  2| 37000|          [26.0,2.0]|
+------+---+---+------+--------------------+



In [13]:
output.columns

['Name', 'Age', 'Exp', 'Salary', 'Independent Features']

In [14]:
finalized_data=output.select("Independent Features","Salary")

In [15]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [25.0,5.0]| 20000|
|          [22.0,1.0]| 50000|
|          [23.0,8.0]| 30000|
|          [27.0,2.0]| 24000|
|          [24.0,7.0]| 25000|
|          [25.0,3.0]| 25000|
|          [24.0,3.0]| 29000|
|          [26.0,2.0]| 37000|
+--------------------+------+



In [16]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [17]:


### Coefficients
regressor.coefficients



DenseVector([-4712.8713, -2772.2772])

In [18]:
### Intercepts
regressor.intercept

154574.257425754

In [19]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [20]:
pred_results.predictions.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,8.0]| 30000|24000.000000000815|
|          [25.0,3.0]| 25000| 28435.64356435614|
|          [26.0,2.0]| 37000|26495.049504949697|
+--------------------+------+------------------+



In [21]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError


(6646.864686468543, 52719210.53491653)